In [3]:
import xml.etree.ElementTree as ET
from lxml import etree
from pathlib import Path
from datetime import datetime
import os, csv, logging, argparse
import pandas as pd
from io import StringIO
import boto3


#dictionary of all the xml files available on the stack exchange data dump site
tables = {
    "Badges.xml": ["Id","UserId","Name","Date"],
    "Comments.xml": ["Id","PostId","Score","Text","CreationDate","UserId"],
    "Posts.xml": ["Id","PostTypeId","ParentId","AcceptedAnswerId","CreationDate","Score","ViewCount","Body","OwnerUserId","LastEditorUserId","LastEditorDisplayName","LastEditDate","LastActivityDate","CommunityOwnedDate","ClosedDate","Title","Tags","AnswerCount","CommentCount","FavoriteCount"],
    "PostHistory.xml": ["Id","PostHistoryTypeId","PostId","RevisionGUID","CreationDate","UserId","UserDisplayName","Comment","Text","CloseReasonId"],
    "PostLinks.xml": ["Id","CreationDate","PostId","RelatedPostId","PostLinkTypeId"],
    "Tags.xml": ["Id","TagName","Count","ExcerptPostId","WikiPostId"],
    "Users.xml": ["Id","Reputation","CreationDate","DisplayName","EmailHash","LastAccessDate","WebsiteUrl","Location","AboutMe","Views","UpVotes","DownVotes","ProfileImageUrl","AccountId"],
    "Votes.xml": ["Id","PostId","VoteTypeId","UserId","CreationDate"]
}


#since my solution leverages s3 and spectrum schema for putting the data 
#in redshift(relational db), hence this function is for connecting to s3 and putting files in the bucket

def put_df_to_s3(bucket_name, entity, data):
    aws_access_key_id= 'AK**************'
    aws_secret_key = 'Qh*************************'
    s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_key)
    s3 = boto3.resource("s3")
    s3_client.put_object(Bucket = bucket_name, Key=f"exchange_data/{entity}/{entity}.csv", Body=data)  
    
#a small useful function for basic cleaning of the data
    
def transform_column(column):
    columnar = column.replace("\r\n","&#xD;&#xA;").replace("\r","&#xD;").replace("\n", "&#xA;")
    return columnar

#a function that loops through the dictionary keys and for each xml file creates the element tree. 
#Since the size of the xml can be a bottle neck, parsing the events is ideal in this case. The data is appended
#to a empty list and then to a dataframe and then converted to csv and put in s3.

def xml_to_df(columns):
    for i in files:
        emp = []
        sourcefilename = '/Users/apuravsharma/Documents/case study outfittery/input/{}'.format(i)
        context = etree.iterparse(sourcefilename, events=('end',), tag='row')
        for event, element in context:
            row=[transform_column(element.attrib[column]) if column in element.attrib else '' for column in columns[i]]
            emp.append(row) 
            df[i] = {i: pd.DataFrame(emp,columns=tables[i])}
            output = StringIO()
            df[i][i].to_csv(output, header=True, float_format='%.16f', index=False)
            put_df_to_s3("test-exchange-study" , i, output.getvalue())
            
#triggering function and time monitoring
            
start_time = datetime.now()
xml_to_df(tables)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))   

